# usage:
0.   update the information in cell 1. very important. make sure this is
correct.
1.   run the first **FOUR** code cells, in order (do not hit "run all" there is broken code below)
2.   open the folder icon on the left bar
3.   download the two files ("A_merge_F25.csv" and "R_merge_F25.csv" or similar)
4.   go to https://massmail.andrew.cmu.edu/ (log in if necessary)
5.   do "Send New MassMail"
6.   run cell 5
6.   fill out the form as follows:
  *   From: stuco@andrew.cmu.edu
  *   Specify mergefile: upload the appropriate CSV from step 3 (A for advisor email, R for recommender)
  *   Subject: see output from code cell 5, should look like "[ACTION REQUIRED] etc" (CHOOSE THE RIGHT ONE)
  *   Reply to: stuco@andrew.cmu.edu (so when they respond, it ends up in stuco inbox)
  *   Bounce Address: you can do either the webmaster (so the failed emails dont end up in official stuco inbox) or whichever officer is running this and is in charge of fixing the bugs
  *   Type Message Below: use the HTML editor (somewhere in the menu bar, symbol looks like the word HTML, very hard to miss) and paste in the appropriate output from cell 5 (starts with `<p>Dear %FIRSTNAME%,</p>`)

At this point you should be good to go.
please use responsibly.

all errors are the user's fault for not checking that this still works as intended 🫵🤣

# Update log:
* code: 2024-09 (working as of 2025-01)
* documentation: 2025-01

# Code

In [ ]:
# 1. data entry for the current semester. change this stuff.

stuco_list_url = "https://docs.google.com/spreadsheets/d/1rijdL1vZbRYvnOla8lpRQPRnvlqtkBHwYvZhn16v4uk"
sem = "Fall 2025"
sem_shortened = "F25"
deadline_d, deadline_t = "Friday, February 14th", "5:00 PM EST"

In [ ]:
# 2. load up the google sheets modules

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
# 3. access the new semester's stuco list sheet. pray that the columns haven't been moved around
#    note: for this to work correctly, the range in the second line should encompass
#          returning instr, instr name/andrewID, advisor name/andrewID, recommender name/andrewID
#    if this is not true, then change the column range so that it does

import pandas as pd, numpy as np
data = gc.open_by_url(stuco_list_url).get_worksheet(0).get("D:P")
df = pd.DataFrame.from_records(data[1:], columns=data[0])


In [ ]:
# 4. read the roster and build the mergefiles
#    builds each list by looking through the unique faculty and packing their
#    associated students into a HTML bulleted list to be inserted into the email

#    just trust the process tbh

def read_roster_new(df):
  header = "\%EMAILADDRESS%,\%FIRSTNAME%,%MINITIAL%,%LASTNAME%,%AFFILIATION%,%DEPTNAME%,\%GRADYEAR%,%CUSTOM1%,%CUSTOM2%,%CUSTOM3%,%CUSTOM4%\n"

  for faculty_type in ["Advisor", "Recommending Faculty"]:
    mergetext = header
    n = 0
    faculty_df = df[f"{faculty_type} AndrewID"].unique()
    for faculty in faculty_df:
      if faculty:
        students = df[df[f"{faculty_type} AndrewID"] == faculty].drop_duplicates("Instructor AndrewID")
        student_names = []
        for s in students.iterrows():
          n += 1
          faculty_name = s[1][f"{faculty_type} Name"]
          faculty_email = s[1][f"{faculty_type} AndrewID"] + "@andrew.cmu.edu"
          student_names.append(f'{s[1]["Instructor First Name"]} {s[1]["Instructor Last Name"]}')
        students_html = build_html_list(student_names)
        mergetext += f"{faculty_email},{faculty_name},,,,,,{students_html},,,\n"

    filename = f"./{faculty_type[0]}_merge_{sem_shortened}.csv"
    with open(filename, "w") as f:
      f.write(mergetext)

def build_html_list(students):
  out = ""
  for student in students:
    if out: out += "</li><li>"
    out += student
  return out

read_roster_new(df)

In [ ]:
# 5. write the email html for each semester

adv_template = f"""
<p>Dear %FIRSTNAME%,</p>
<p>The Student College Executive Committee is currently reviewing applications
from students applying to teach a StuCo (student-taught course) in the {sem}
semester.<br /> One of your advisees has applied; the official StuCo policy
states that each time for a student to teach a StuCo, their academic advisor
must provide their approval.<br /> This is a list of your advisees who indicated
that they plan to teach a StuCo in the {sem} semester. Please let us know,
for each student, if you approve them to teach a StuCo <strong>by {deadline_t}
on {deadline_d}</strong>.<br /> Thank you!</p>
<p><br />List:</p>
<ul>
<li>%CUSTOM1%</li>
</ul>
<p>Best regards, <br />StuCo Exec Committee</p>
"""
adv_title = "[ACTION REQUIRED] Academic Advisor Approval Required for Potential StuCo Instructor"
rec_template = f"""
<p>Dear %FIRSTNAME%,</p>
<p> The Student College Executive Committee is currently reviewing applications
from students applying to teach a StuCo (Student-taught course) in the {sem}
semester.<br/> The official StuCo policy states that for a student to teach a
StuCo, a current faculty member at CMU must endorse the instructor.<br/> This is
a list of students who indicated that you will be their recommending faculty
member. Please let us know, for each student, if you believe them to be capable
of teaching their own StuCo <strong>by {deadline_t} on {deadline_d}</strong>.
<br/> Thank you! </p>
<br/>List:
<ul>
<li>%CUSTOM1%</li>
</ul>
<p>Best regards, <br />StuCo Exec Committee</p>
"""
rec_title = "[ACTION REQUIRED] Recommending Faculty Endorsement Required for Potential StuCo Instructor"

for t in ((adv_title, adv_template), (rec_title, rec_template)): print(t[0]); print(t[1])

[ACTION REQUIRED] Academic Advisor Approval Required for Potential StuCo Instructor

<p>Dear %FIRSTNAME%,</p>
<p>The Student College Executive Committee is currently reviewing applications
from students applying to teach a StuCo (student-taught course) in the Fall 2025
semester.<br /> One of your advisees has applied; the official StuCo policy
states that each time for a student to teach a StuCo, their academic advisor
must provide their approval.<br /> This is a list of your advisees who indicated
that they plan to teach a StuCo in the Fall 2025 semester. Please let us know,
for each student, if you approve them to teach a StuCo <strong>by 5:00 PM EST
on Friday, February 14th</strong>.<br /> Thank you!</p>
<p><br />List:</p>
<ul>
<li>%CUSTOM1%</li>
</ul>
<p>Best regards, <br />StuCo Exec Committee</p>

[ACTION REQUIRED] Recommending Faculty Endorsement Required for Potential StuCo Instructor

<p>Dear %FIRSTNAME%,</p>
<p> The Student College Executive Committee is currently reviewing a

In [ ]:
!cat R_merge_F25.csv

\%EMAILADDRESS%,\%FIRSTNAME%,%MINITIAL%,%LASTNAME%,%AFFILIATION%,%DEPTNAME%,\%GRADYEAR%,%CUSTOM1%,%CUSTOM2%,%CUSTOM3%,%CUSTOM4%
jmackey@andrew.cmu.edu,John Mackey,,,,,,Aidan Pevida</li><li>Allen Yang,,,
blawrenc@andrew.cmu.edu,Betsy Lawrence,,,,,,Anthony Cheng,,,
tamal@andrew.cmu.edu,Tamal Mukherjee,,,,,,Fiona Fisher,,,
agr@andrew.cmu.edu,Anthony Rowe,,,,,,Matthew Kibarian</li><li>Gary (Sky) Bailey,,,
wnace@andrew.cmu.edu,Bill Nace,,,,,,Nick Yaeger,,,
rwhipple@andrew.cmu.edu,R. James Whipple ,,,,,,Zhengkang Lyu,,,
sfinger@andrew.cmu.edu,Susan Finger,,,,,,Marina Wang,,,
sandage@andrew.cmu.edu,Scott Sandage,,,,,,Ayush Gupta,,,
ihansen@andrew.cmu.edu,Isla Hansen,,,,,,Ada Menger-Thau</li><li>Annie Marcelino,,,
dhx5@andrew.cmu.edu,Deanna Matthews,,,,,,Vionnie Khong,,,
lohp@andrew.cmu.edu,Po-Shen Loh,,,,,,Andrew Peng</li><li>John Ternullo,,,
pegden@andrew.cmu.edu,Wesley Pegden,,,,,,Henry Siegel,,,
sraja@andrew.cmu.edu,Raja Sooramorthy,,,,,,Ayush Guraria,,,
vferrerm@andrew.cmu.edu,Veronika Eb

# DO NOT RUN ANYTHING BELOW HERE (it's all stuff dealing with renaming contracts and broken 🪦)

In [ ]:
df.head()

,Course Number,Short Title,Long Title,Returning Instructor,Instructor First Name,Instructor Last Name
0,98-002,STUCO: MAST CONNECT4,Mastering Connect 4,TRUE,Aidan,Pevida
1,98-008,STUCO: RUSTLANG,Intro to Rust Lang,TRUE,Connor,Tsui
2,98-010,STUCO: INTRO TO BJJ,Intro to Submission Grappling,FALSE,Mateus,Gondinho
3,98-010,STUCO: INTRO TO BJJ,Intro to Submission Grappling,FALSE,Kai,Fergerstrom
4,98-012,STUCO: FUN WTH ROBTS,Fun with Robots,TRUE,Jordan,Petzold


In [ ]:
# make contract names
data = gc.open_by_url(stuco_list_url).get_worksheet(0).get("A:F")
df = pd.DataFrame.from_records(data[1:], columns=data[0])

In [ ]:
for instr in df.iterrows():
  i = instr[1]
  contract_title = f'{sem_shortened} Instructor Contract_{i["Course Number"]} {i["Long Title"]}_{i["Instructor First Name"]} {i["Instructor Last Name"]}'
  print(contract_title)

S25 Instructor Contract_98-002 Mastering Connect 4_Aidan Pevida
S25 Instructor Contract_98-008 Intro to Rust Lang_Connor Tsui
S25 Instructor Contract_98-010 Intro to Submission Grappling_Mateus Gondinho
S25 Instructor Contract_98-010 Intro to Submission Grappling_Kai Fergerstrom
S25 Instructor Contract_98-012 Fun with Robots_Jordan Petzold
S25 Instructor Contract_98-012 Fun with Robots_Kailey Hua
S25 Instructor Contract_98-016 Genshin Impact Theorycrafting 101_Marina Wang
S25 Instructor Contract_98-016 Genshin Impact Theorycrafting 101_Zhengkang (Allan) Lyu
S25 Instructor Contract_98-040 Sew Sustainable_Katrina Darms
S25 Instructor Contract_98-043 Chess Tactics and Strategy_Andrew Peng
S25 Instructor Contract_98-043 Chess Tactics and Strategy_Ethan Gu
S25 Instructor Contract_98-043 Chess Tactics and Strategy_Vionnie Khong
S25 Instructor Contract_98-076 Fire Emblem Design and Analysis_Shima McClure
S25 Instructor Contract_98-076 Fire Emblem Design and Analysis_Andrew Zhou
S25 Instructor